In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11482,2021-08-23T17:00:00,ITA,19,Sicilia,38.115697,13.362357,729,88,817,23329,...,deceduti: n. 7 del 22/08/21 - n. 7 del 21/08/2...,12.0,NaN,NaN,265673.0,0.0,2924007.0,2694200.0,ITG,ITG1
11483,2021-08-23T17:00:00,ITA,9,Toscana,43.769231,11.255889,366,41,407,11790,...,NaN,3.0,NaN,NaN,262586.0,4199.0,4236438.0,1539109.0,ITI,ITI1
11484,2021-08-23T17:00:00,ITA,10,Umbria,43.106758,12.388247,53,7,60,1827,...,"Si fa presente che 1 dei ricoveri NON UTI, non...",1.0,NaN,NaN,60876.0,0.0,1062826.0,640921.0,ITI,ITI2
11485,2021-08-23T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,5,0,5,142,...,NaN,0.0,NaN,NaN,11210.0,772.0,105575.0,57113.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11482,2021-08-23T17:00:00,19,Sicilia,729,88,817,23329,24146,686,1121,...,265673,5618207,2133932.0,12.0,265673.0,0.0,2924007.0,2694200.0,ITG,ITG1
11483,2021-08-23T17:00:00,9,Toscana,366,41,407,11790,12197,-68,434,...,266785,5775547,2638370.0,3.0,262586.0,4199.0,4236438.0,1539109.0,ITI,ITI1
11484,2021-08-23T17:00:00,10,Umbria,53,7,60,1827,1887,-71,7,...,60876,1703747,420584.0,1.0,60876.0,0.0,1062826.0,640921.0,ITI,ITI2
11485,2021-08-23T17:00:00,2,Valle d'Aosta,5,0,5,142,147,0,0,...,11982,162688,78204.0,0.0,11210.0,772.0,105575.0,57113.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-08-23', '2021-08-22')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-08-23T17:00:00,1,Piemonte,143,12,155,3427,3582,-61,102,...,370660,5856179,2201913.0,1.0,351062.0,19598.0,3300194.0,2555985.0,ITC,ITC1
1,2021-08-23T17:00:00,2,Valle d'Aosta,5,0,5,142,147,0,0,...,11982,162688,78204.0,0.0,11210.0,772.0,105575.0,57113.0,ITC,ITC2
2,2021-08-23T17:00:00,3,Lombardia,320,43,363,11938,12301,55,200,...,865362,13253673,5076404.0,4.0,805062.0,60300.0,10149009.0,3104664.0,ITC,ITC4
3,2021-08-23T17:00:00,5,Veneto,186,48,234,12605,12839,176,331,...,449794,10326391,2002227.0,2.0,436540.0,13254.0,6259639.0,4066752.0,ITH,ITH3
4,2021-08-23T17:00:00,6,Friuli Venezia Giulia,34,11,45,866,911,-29,25,...,109913,2429978,744368.0,3.0,95027.0,14886.0,1976685.0,453293.0,ITH,ITH4
5,2021-08-23T17:00:00,7,Liguria,84,11,95,1931,2026,35,57,...,108599,2058400,786575.0,0.0,108599.0,0.0,1521074.0,537326.0,ITC,ITC3
6,2021-08-23T17:00:00,8,Emilia-Romagna,402,52,454,14278,14732,263,558,...,407449,7778869,2004424.0,5.0,407096.0,353.0,5440760.0,2338109.0,ITH,ITH5
7,2021-08-23T17:00:00,9,Toscana,366,41,407,11790,12197,-68,434,...,266785,5775547,2638370.0,3.0,262586.0,4199.0,4236438.0,1539109.0,ITI,ITI1
8,2021-08-23T17:00:00,10,Umbria,53,7,60,1827,1887,-71,7,...,60876,1703747,420584.0,1.0,60876.0,0.0,1062826.0,640921.0,ITI,ITI2
9,2021-08-23T17:00:00,11,Marche,44,13,57,2889,2946,10,34,...,109115,1427407,830921.0,3.0,109115.0,0.0,1260677.0,166730.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-08-22T17:00:00,1,Piemonte,140,12,152,3491,3643,62,177,...,370558,5845437,2199694.0,0.0,350982.0,19576.0,3297538.0,2547899.0,ITC,ITC1
1,2021-08-22T17:00:00,2,Valle d'Aosta,5,0,5,142,147,-6,7,...,11982,162424,78105.0,0.0,11210.0,772.0,105570.0,56854.0,ITC,ITC2
2,2021-08-22T17:00:00,3,Lombardia,311,41,352,11894,12246,114,406,...,865162,13243184,5073896.0,1.0,804900.0,60262.0,10143418.0,3099766.0,ITC,ITC4
3,2021-08-22T17:00:00,5,Veneto,187,48,235,12428,12663,-31,420,...,449463,10316060,2001042.0,5.0,436208.0,13255.0,6255258.0,4060802.0,ITH,ITH3
4,2021-08-22T17:00:00,6,Friuli Venezia Giulia,32,9,41,899,940,2,78,...,109888,2428502,744038.0,1.0,95006.0,14882.0,1975603.0,452899.0,ITH,ITH4
5,2021-08-22T17:00:00,7,Liguria,78,11,89,1902,1991,4,162,...,108542,2055977,785684.0,0.0,108542.0,0.0,1519818.0,536159.0,ITC,ITC3
6,2021-08-22T17:00:00,8,Emilia-Romagna,387,47,434,14035,14469,273,555,...,406893,7765652,2002774.0,4.0,406540.0,353.0,5435080.0,2330572.0,ITH,ITH5
7,2021-08-22T17:00:00,9,Toscana,349,43,392,11873,12265,0,594,...,266351,5768964,2635378.0,4.0,262176.0,4175.0,4231180.0,1537784.0,ITI,ITI1
8,2021-08-22T17:00:00,10,Umbria,52,6,58,1900,1958,-51,116,...,60869,1702292,420426.0,2.0,60869.0,0.0,1062297.0,639995.0,ITI,ITI2
9,2021-08-22T17:00:00,11,Marche,45,10,55,2881,2936,-42,140,...,109081,1426798,830647.0,1.0,109081.0,0.0,1260147.0,166651.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-08-23T17:00:00,1,Piemonte,143,12,155,3427,3582,-61,102,...,5856179,2201913.0,1.0,351062.0,19598.0,3300194.0,2555985.0,ITC,ITC1,4.341375e+06
1,2021-08-23T17:00:00,2,Valle d'Aosta,5,0,5,142,147,0,0,...,162688,78204.0,0.0,11210.0,772.0,105575.0,57113.0,ITC,ITC2,1.255010e+05
2,2021-08-23T17:00:00,3,Lombardia,320,43,363,11938,12301,55,200,...,13253673,5076404.0,4.0,805062.0,60300.0,10149009.0,3104664.0,ITC,ITC4,1.010397e+07
3,2021-08-23T17:00:00,5,Veneto,186,48,234,12605,12839,176,331,...,10326391,2002227.0,2.0,436540.0,13254.0,6259639.0,4066752.0,ITH,ITH3,4.907704e+06
4,2021-08-23T17:00:00,6,Friuli Venezia Giulia,34,11,45,866,911,-29,25,...,2429978,744368.0,3.0,95027.0,14886.0,1976685.0,453293.0,ITH,ITH4,1.211357e+06
5,2021-08-23T17:00:00,7,Liguria,84,11,95,1931,2026,35,57,...,2058400,786575.0,0.0,108599.0,0.0,1521074.0,537326.0,ITC,ITC3,1.543127e+06
6,2021-08-23T17:00:00,8,Emilia-Romagna,402,52,454,14278,14732,263,558,...,7778869,2004424.0,5.0,407096.0,353.0,5440760.0,2338109.0,ITH,ITH5,4.467118e+06
7,2021-08-23T17:00:00,9,Toscana,366,41,407,11790,12197,-68,434,...,5775547,2638370.0,3.0,262586.0,4199.0,4236438.0,1539109.0,ITI,ITI1,3.722729e+06
8,2021-08-23T17:00:00,10,Umbria,53,7,60,1827,1887,-71,7,...,1703747,420584.0,1.0,60876.0,0.0,1062826.0,640921.0,ITI,ITI2,8.802850e+05
9,2021-08-23T17:00:00,11,Marche,44,13,57,2889,2946,10,34,...,1427407,830921.0,3.0,109115.0,0.0,1260677.0,166730.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-08-23T17:00:00,1,Piemonte,143,12,155,3427,3582,-61,102,...,2555985.0,ITC,ITC1,4.341375e+06,Piemonte,143,5824,12,628,99
1,2021-08-23T17:00:00,2,Valle d'Aosta,5,0,5,142,147,0,0,...,57113.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,5,83,0,33,0
2,2021-08-23T17:00:00,3,Lombardia,320,43,363,11938,12301,55,200,...,3104664.0,ITC,ITC4,1.010397e+07,Lombardia,320,6369,43,1416,114
3,2021-08-23T17:00:00,5,Veneto,186,48,234,12605,12839,176,331,...,4066752.0,ITH,ITH3,4.907704e+06,Veneto,186,6000,48,1000,0
4,2021-08-23T17:00:00,6,Friuli Venezia Giulia,34,11,45,866,911,-29,25,...,453293.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,34,1277,11,175,0
5,2021-08-23T17:00:00,7,Liguria,84,11,95,1931,2026,35,57,...,537326.0,ITC,ITC3,1.543127e+06,Liguria,84,1684,11,218,12
6,2021-08-23T17:00:00,8,Emilia-Romagna,402,52,454,14278,14732,263,558,...,2338109.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,402,6538,52,760,0
7,2021-08-23T17:00:00,9,Toscana,366,41,407,11790,12197,-68,434,...,1539109.0,ITI,ITI1,3.722729e+06,Toscana,366,5033,41,570,40
8,2021-08-23T17:00:00,10,Umbria,53,7,60,1827,1887,-71,7,...,640921.0,ITI,ITI2,8.802850e+05,Umbria,53,684,7,91,36
9,2021-08-23T17:00:00,11,Marche,44,13,57,2889,2946,10,34,...,166730.0,ITI,ITI3,1.518400e+06,Marche,44,951,13,213,61


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-08-23T17:00:00,19,Sicilia,729,88,817,23329,24146,686,1121,...,9.44,657,-229,12565,5209.0,20,415,8.92,0.02256,5.35
1,2021-08-23T17:00:00,8,Emilia-Romagna,402,52,454,14278,14732,263,558,...,6.84,243,3,13217,1650.0,3,290,4.22,0.01249,9.12
2,2021-08-23T17:00:00,9,Toscana,366,41,407,11790,12197,-68,434,...,7.19,-83,-160,6583,2992.0,3,499,6.59,0.01166,7.17
3,2021-08-23T17:00:00,12,Lazio,538,67,605,16472,17077,-281,348,...,7.10,-311,-106,0,0.0,8,621,inf,0.00593,6.33
4,2021-08-23T17:00:00,5,Veneto,186,48,234,12605,12839,176,331,...,4.80,177,-89,10331,1185.0,0,155,3.20,0.00674,9.17
5,2021-08-23T17:00:00,20,Sardegna,197,22,219,7674,7893,48,311,...,10.78,43,55,2446,1463.0,1,262,12.71,0.01907,4.27
6,2021-08-23T17:00:00,18,Calabria,123,8,131,3925,4056,165,257,...,4.73,164,44,2273,1708.0,3,89,11.31,0.01335,3.93
7,2021-08-23T17:00:00,3,Lombardia,320,43,363,11938,12301,55,200,...,3.04,44,-206,10489,2508.0,0,145,1.91,0.00198,8.56
8,2021-08-23T17:00:00,15,Campania,361,23,384,8986,9370,-164,190,...,4.20,-189,-280,14671,1429.0,3,351,1.30,0.00328,7.64
9,2021-08-23T17:00:00,16,Puglia,186,22,208,4423,4631,6,144,...,4.56,-5,-117,7286,1775.0,0,138,1.98,0.00359,6.53


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Sardegna,22,182,204,10.78
1,Sicilia,88,844,932,9.44
2,Umbria,7,84,91,7.69
3,Abruzzo,13,164,177,7.34
4,Toscana,41,529,570,7.19
5,Lazio,67,876,943,7.10
6,Emilia-Romagna,52,708,760,6.84
7,Friuli Venezia Giulia,11,164,175,6.29
8,Marche,13,200,213,6.10
9,Liguria,11,207,218,5.05


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))